# 1) Init ESML 

In [1]:
sys.path.append(os.path.abspath("../common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject
from azureml.core import Workspace

p = ESMLProject() # Makes it "environment aware (dev,test,prod)", and "configuration aware"
ws = p.get_workspace_from_config()
p.describe()

Training

 - ds01_diabetes
master/1_projects/project002/03_diabetes_model_reg/train/ds01_diabetes/in/dev/2020/01/01/
master/1_projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/bronze/dev/
master/1_projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/silver/dev/

 - ds02_other
master/1_projects/project002/03_diabetes_model_reg/train/ds02_other/in/dev/2020/01/01/
master/1_projects/project002/03_diabetes_model_reg/train/ds02_other/out/bronze/dev/
master/1_projects/project002/03_diabetes_model_reg/train/ds02_other/out/silver/dev/
Training GOLD 

master/1_projects/project002/03_diabetes_model_reg/train/gold/dev/
 

ENVIRONMENT - DEV, TEST, or PROD?
ACTIVE ENVIRONMENT = dev
ACTIVE subscription = ca0a8c40-b06a-4e4e-8434-63c03a1dee34
- MSFT-WEU-EAP_PROJECT02_AI-DEV-RG
- msft-weu-DEV-eap-proj02_ai-amls
- westeurope
- MSFT-WEU-EAP_CMN_AI-DEV-RG
- msft-weu-dev-cmnai-vnet
- msft-weu-dev-cmnai-sn-aml


In [2]:
unregister_all_datasets=False

In [3]:
if(unregister_all_datasets):
    p.unregister_all_datasets(ws) # For DEMO purpose

p.init(ws) 
p.dev_test_prod = "dev"
#Note: This happens in the INIT() method of DriverAPI/CallingAppliation...not loading for every Scoring

...
Using GEN2 as Datastore
ds01_diabetes
ds02_other

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 2 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002lake 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files
Dataset 'ds02_other' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN_CSV, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


# 2) Test an `existing` Model webservice (AKS)?

### 2a) Lets simulate INFERENCE: data arives from CLIENT to a `DriverAPI` that itself will call `this ModelAPI` (AKS)
Note: `Step 1 and 2 are out of scope` of from the MACHINE LEARNING solution (and ESML), other than...
 - `ESML` being the creator of AML pipeline - `Bronze_2_Gold` that you can reuse
 - `ESML lakedesign`, that should be used, if INFERENCE are to be stored/cached in the lake

NOW - the scenario: 

- DriverAPI will:

    - 1)Fetch missing datasources from different systems, to get all data sources ds01, ds02,ds03 & SAVE to INFERENCE/IN folder in the datalake
        - Use the `ESML` lakedesign:  IF `inference_model_version=1` in `settings/project_specific/lake_settings.json` then...
        - `ESML` will switch to write at `INFERENCE/v1/ds01/IN/2020/01/01` folder 
    - 2)Call `Bronze_2_Gold` batch pipeline (AML) - same pipeline that was used for training: `IN->Bronze->Silver->Gold`, except...
        - `Bronze_2_Gold` pipeline will read from IN folder, but configured for "INFERENCE", read/write to `"mirror"` place in the datalake, that will `CACHE` online predicted results
        - (LABEL values are missing of course)
    - 3) `DriverAPI calls ModelAPI` (This AKS webservuce) with data to score -> `X_text`

- `ESML` ModelAPI will:
    - 4) `Score` the data, and return results, also `save the results to the datalake`
        - if `p.rnd=True` nothing is stored.
    - 5) `Connect the scored data` to a caller, and the individual scores to a identity (user/machine)
        - GLOBALLY & ESML Managed: The scoring file gets a caller_id. 
            - You: Need to pass the called-guid as a parameter. 
                - Alternatively, you can have a column in the dataframe called `esml_caller_id_string`, and ESML will use that.
        - LOCALLY: The rows in the dataframe has an identity. 
            - You: Here you need to have a column in the dataframe, such as `user_id`, to be able to connect each row to a scoring.


Step 1 and 2 are out of scope of from the MACHINE LEARNING solution  
We START this notebooke at step 3 - we have `X_test` in `GOLD_Validate` status 

In [4]:
import datetime as dt
now = dt.datetime.now()
folder = now.strftime('%Y_%m_%d') 
print(folder)

2021_04_10


In [5]:
if(unregister_all_datasets): # Create a GOLD dataset, and SPLIT it
    df_01 = p.DatasetByName("ds01_diabetes").Silver.to_pandas_dataframe()
    ds_gold_v1 = p.save_gold(df_01)

    label = "Y"
    train_6, validate_set_2, test_set_2 = p.split_gold_3(0.6, label)

In [6]:
# 2) Bronze_2_Gold done, we can fetch X_test
X_test, y_test, tags = p.get_gold_validate_Xy() # Version is default latest
print(tags)

M03_GOLD_VALIDATE : (37, 11)
X_test  (37, 10)
y_test  (37,)
{'split_percentage': '0.2', 'label': 'Y'}


In [7]:
X_test.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,0.048974,0.050680,0.123131,0.083844,-0.104765,-0.100895,-0.069172,-0.002592,0.036646,-0.030072
1,0.070769,-0.044642,0.069241,0.037939,0.021822,0.001504,-0.036038,0.039106,0.077633,0.106617
2,0.056239,0.050680,-0.030996,0.008101,0.019070,0.021233,0.033914,-0.039493,-0.029528,-0.059067
3,0.016281,-0.044642,0.017506,-0.022885,0.060349,0.044406,0.030232,-0.002592,0.037232,-0.001078
4,0.023546,-0.044642,0.110198,0.063187,0.013567,-0.032942,-0.024993,0.020655,0.099240,0.023775


### 2b) Call onlin ModelAPI - score 1 row (alt 1)

import pandas as pd
#3) `DriverAPI calls ModelAPI`

keyvault = p.ws.get_default_keyvault()
api_url = keyvault.get_secret(name="esml-dev-p02-m03-api")
api_key = keyvault.get_secret(name="esml-dev-p02-m03-apisecret")  # esml-dev-p02-m03-apisecret

df = ESMLProject.call_webservice_own_url(X_test, api_url,api_key) # rest call. STATIC method - simulate "we dont need ESML for this" - just an URL and KEY from DriverAPI
df.head() # Print scoring

### 2b) Call online ModelAPI - score all rows (alt 2) - `via ESML wrapper`
- ESML benefits: Has built in `logic` to save scored_results, to unique folders during the day.
- Note: We can also save a GUID/ID, `to see which CALLER/User-Guid the scoring is about` 
    - We can have a User_id_GUID as a "feature/column" in X_test, or as a parameter in the call

In [8]:
p.inferenceModelVersion=1
print(p.ScoredPath)

master/1_projects/project002/03_diabetes_model_reg/inference/1/scored/dev/


In [9]:
caller_user_id = '81965d9c-40ca-4e47-9723-5a608a32a0e4' # Connect the scoring to a caller/user, globally for all rows

df = p.call_webservice(p.ws, X_test, caller_user_id, False) # Auto-fetch key from keyvault, 1stRowOnlye=False
df.head()

Note: USING enterprise performance settings. (This can be overridden in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=True
Note: Fetching keys automatically via workspace keyvault.
Saving scoring to lake for project folder project002 and inference_model_version: 1 ...
...
..

Saved DATA to score successfully in LAKE, as file 'to_score_81965d9c-40ca-4e47-9723-5a608a32a0e4.parquet'
..
Saved SCORED data in LAKE, as file 'scored_81965d9c-40ca-4e47-9723-5a608a32a0e4.parquet'


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,result
0,0.048974,0.050680,0.123131,0.083844,-0.104765,-0.100895,-0.069172,-0.002592,0.036646,-0.030072,243.130371
1,0.070769,-0.044642,0.069241,0.037939,0.021822,0.001504,-0.036038,0.039106,0.077633,0.106617,293.235993
2,0.056239,0.050680,-0.030996,0.008101,0.019070,0.021233,0.033914,-0.039493,-0.029528,-0.059067,94.014447
3,0.016281,-0.044642,0.017506,-0.022885,0.060349,0.044406,0.030232,-0.002592,0.037232,-0.001078,148.809757
4,0.023546,-0.044642,0.110198,0.063187,0.013567,-0.032942,-0.024993,0.020655,0.099240,0.023775,252.999922


# 3) How to GET SCORE data? how to FILTER scored results?

In [10]:
X_test, y_test, tags = p.get_gold_validate_Xy()
caller_user_id = '91965d9c-40ca-4e47-9723-5a608a32a0e4' # Connect the scoring to a caller/user, globally for all rows

df = p.call_webservice(p.ws, X_test, caller_user_id, False) # Auto-fetch key from keyvault, 1stRowOnlye=False
df.head()

M03_GOLD_VALIDATE : (37, 11)
X_test  (37, 10)
y_test  (37,)
Note: USING enterprise performance settings. (This can be overridden in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=True
Note: Fetching keys automatically via workspace keyvault.
Saving scoring to lake for project folder project002 and inference_model_version: 1 ...
...

Saved DATA to score successfully in LAKE, as file 'to_score_91965d9c-40ca-4e47-9723-5a608a32a0e4.parquet'
Saved SCORED data in LAKE, as file 'scored_91965d9c-40ca-4e47-9723-5a608a32a0e4.parquet'


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,result
0,0.048974,0.050680,0.123131,0.083844,-0.104765,-0.100895,-0.069172,-0.002592,0.036646,-0.030072,243.130371
1,0.070769,-0.044642,0.069241,0.037939,0.021822,0.001504,-0.036038,0.039106,0.077633,0.106617,293.235993
2,0.056239,0.050680,-0.030996,0.008101,0.019070,0.021233,0.033914,-0.039493,-0.029528,-0.059067,94.014447
3,0.016281,-0.044642,0.017506,-0.022885,0.060349,0.044406,0.030232,-0.002592,0.037232,-0.001078,148.809757
4,0.023546,-0.044642,0.110198,0.063187,0.013567,-0.032942,-0.024993,0.020655,0.099240,0.023775,252.999922


In [11]:
import datetime as dt
now = dt.datetime.now()
date_filter = now.strftime('%Y_%m_%d') 
print(date_filter)

2021_04_10


In [12]:
ds_list, df_all = p.get_scored(date_filter, "10",'81965d9c-40ca-4e47-9723-5a608a32a0e4')

In [13]:
#df = p.get_scored('last_month')
ds_list1, df_all1 = p.get_scored(date_filter, "1",'81965d9c-40ca-4e47-9723-5a608a32a0e4')
print("Datasets found in filter", len(ds_list1))
print("Example: How many rows in 1st dataset? ", ds_list1[0].to_pandas_dataframe().shape[0])
print("All rows", df_all1.shape[0])

df_all1.head()

Datasets found in filter 1
Example: How many rows in 1st dataset?  37
All rows 37


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,result
0,0.048974,0.050680,0.123131,0.083844,-0.104765,-0.100895,-0.069172,-0.002592,0.036646,-0.030072,243.130371
1,0.070769,-0.044642,0.069241,0.037939,0.021822,0.001504,-0.036038,0.039106,0.077633,0.106617,293.235993
2,0.056239,0.050680,-0.030996,0.008101,0.019070,0.021233,0.033914,-0.039493,-0.029528,-0.059067,94.014447
3,0.016281,-0.044642,0.017506,-0.022885,0.060349,0.044406,0.030232,-0.002592,0.037232,-0.001078,148.809757
4,0.023546,-0.044642,0.110198,0.063187,0.013567,-0.032942,-0.024993,0.020655,0.099240,0.023775,252.999922


In [14]:
ds_list, df_all = p.get_scored(date_filter, "1")
print("Datasets found in filter", len(ds_list))
print("Example: How many rows in 1st dataset? ", ds_list[0].to_pandas_dataframe().shape[0])
print("All rows", df_all.shape[0])

Datasets found in filter 2
Example: How many rows in 1st dataset?  37
All rows 74


 ** Train model & register ....this happended already in another notebook/MLops pipline step
> model = p.register_active_model()  
> print(model.name, model.description, model.version)  


# 3a) ESML `Deploy a new ONLINE` webservice (AKS)
- Deploy "offline" from old `AutoML run` for `DEV` environment
- To →  `DEV`, `TEST` or `PROD` environment
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling`
    - Tip: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

In [ ]:
inference_config, model, best_run = p.get_active_model_inference_config(ws) #  AutoML support 
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config, True) # overwrite_endpoint=True

# Test the NEW AKS WebService

In [ ]:
X_test, y_test, tags = p.get_gold_validate_Xy() # Version is default latest
print(tags)

In [ ]:
X_test.head()

In [ ]:
df = p.call_webservice(p.ws, X_test) # Auto-fetch key from keyvault, 1stRowOnlye=False  |   p.call_webservice(p.ws, X_test, caller_user_id, False)
df.head()

# Code to `embed in YOUR DriverApi` - to call this webservice `without ESML`
- `NB!` This will `NOT cache` the results automatically, since not going via the ESML SDK. You need to save the scoring yourself ( if this is needed)
    - ESML benefits: Has built in `logic` to save scored_results, to unique folders during the day.
            # Note: We can also save a GUID/ID, for which CALLER/User-Guid it is about. We can have a User_id_GUID as a "feature/column" in X_test, or as a parameter

In [ ]:
def call_webservice_code(rows, api_uri,api_key, allowSelfSigned=True):
    # bypass the server certificate verification on client side
    if allowSelfSigned and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

    X_test_json_works = json.dumps({'data': rows.to_dict(orient='records')})

    headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + api_key}
    resp = requests.post(api_uri, X_test_json_works , headers=headers) 

    # Here you can pass forward the response, save it to the datalake, or as below return a PANDAS dataframe
    res_dict = json.loads(resp.text)
    res_dict_ast = ast.literal_eval(res_dict)
    return pd.read_json(res_dict) # to pandas

## 3b) ESML `Deploy BATCH` pipeline
- Deploy same model "offline / previous" `AutoML Run` for `DEV` environment
- To →  `DEV`, `TEST` or `PROD` environment


# 5a alt2) If not using AutoML - you need to manuallt create `environemnt + entryscript + inference config`
https://github.com/Azure/MachineLearningNotebooks/blob/bda592a236eaf2dbc54b394e1fa1b539e0297908/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.ipynb

In [ ]:
# Create ENVIRONMENT - If not AUTOML
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda_deps = CondaDependencies.create(conda_packages=['numpy','scikit-learn==0.19.1','scipy'], pip_packages=['azureml-defaults', 'inference-schema'])
myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

In [ ]:
%%writefile score.py
import os
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [ ]:
from azureml.core.model import InferenceConfig
inf_config = InferenceConfig(entry_script='score.py', environment=myenv)

# OTHER - What more can you retrieve from AutoMLFactory and ESMLProject?

In [ ]:
from baselayer_azure_ml import AutoMLFactory
from baselayer_azure_ml import ComputeFactory

target_model, target_best_run_id = AutoMLFactory().get_latest_model(ws)
print(target_model.name)
print(target_best_run_id)

run, exp = p.get_active_model_run_and_experiment()
inference_config = p.get_active_model_inference_config()
print(run)
print(exp.name)